In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sb
import random

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics

%matplotlib inline

In [2]:
origins = ["DR_Congo", "Afghanistan", "Syria", "Myanmar", "Sudan"]
destinations = ["USA", "UK", "France", "Canada", "Italy", "Germany"]

org_codes = ["COD", "AFG", "SYR", "MMR", "SDN"]
dest_codes = ["USA", "GBR", "FRA", "CAN", "ITA","DEU"]

years = [2000 + i for i in range(19)]

features = [
 'applied',
 'accepted',
 'Rejected',
 'decisions',
 'month_number',
 'month_id',
 'deaths',
 'last_month',
 'two_months_ago',
 'distance',
 'origin_gdp',
 'dest_gdp']

responses = ['Value', 'next_month', 'two_months_later']

all_variables = features + responses

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sept", "Oct", "Nov", "Dec"]


In [3]:
coordinates = {}
coordinates["Syria"] = (34.8, 38.997)
coordinates["DR_Congo"] = (-4.0383, 21.759)
coordinates["Afghanistan"] = (33.939, 67.71)
coordinates["Sudan"] = (12.863, 30.218)
coordinates["Myanmar"] = (21.9162, 95.956)

coordinates["USA"] = (37.09, -95.713)
coordinates["UK"] = (55.378, -3.436)
coordinates["France"] = (46.228, 2.214)
coordinates["Germany"] = (51.166, 10.4515)
coordinates["Italy"] = (41.872, 12.5674)
coordinates["Canada"] = (56.1304, -106.3468)


dest_longs = [coordinates[dest][1] for dest in destinations]
dest_lats = [coordinates[dest][0] for dest in destinations]

org_longs = [coordinates[org][1] for org in origins]
org_lats = [coordinates[org][0] for org in origins]

In [4]:
def get_metrics(y_test, y_pred):

    """Gets the metrics of the fit"""

    if min(y_pred) < 0:
#         print(min(y_pred))
        y_pred += abs(min(y_pred))
    r2 = metrics.r2_score(y_test, y_pred)
    r2 = r2.__round__(4)
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    rmse = rmse.__round__(4)

    return r2, rmse

In [5]:
def make_new_features(df):
    
    '''Makes new features for grouped dataframe'''
    
    for month in months:
        df.ix[df.Month.str.contains(month), ['Month']] = month
    month_ids = []
    d_lats = []
    d_longs = []
    o_lats = []
    o_longs = []
    for index, row in df.iterrows():
        month = row.Month
        month_ids.append(months.index(month) + 1)
        d_lats.append(coordinates[row.destination][0])
        o_lats.append(coordinates[row.Origin][0])
        d_longs.append(coordinates[row.destination][1])
        o_longs.append(coordinates[row.Origin][1])
    df['month_id'] = month_ids
    df['month_number'] =12*(df['Year']-2000) + df['month_id']
    df['dest_lat'] = d_lats
    df['dest_long'] = d_longs
    df['org_lat'] = o_lats
    df['org_long'] = o_longs
    df = df.sort_values(by='month_number')
    
    return df

In [6]:
def get_distance(o_coords, d_coords, length):
    '''Gets the distance between two countries using Havrsine formula'''
    # Earth's radius in km
    R = 6371.
    
    d_lat, d_long = np.deg2rad(d_coords[0]), np.deg2rad(d_coords[1])
    o_lat, o_long = np.deg2rad(o_coords[0]), np.deg2rad(o_coords[1])
    
    a = (np.sin((d_lat - o_lat)/2))**2 + \
        np.cos(d_lat)*np.cos(o_lat)*(np.sin((d_long - o_long)/2))**2
        
    c = 2*np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = R*c
    
    return [d for i in range(length)]

In [7]:
def hyper_search(X, y, model, n_tests = 2):
    '''Searches for appropriate hyperparameters by grid cross validation using RMSE as metric'''
    
    rmses = {}
    for i in range(n_tests):
        rmses[i] = {}
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=i+1)
        
        y_test_log, y_train_log = y_test, y_train
        
        if model != 'LinearRegression' and model != 'BoostedTree':
            grid = [j for j in range(hyper_p_grids[model][0], hyper_p_grids[model][1], \
                                     hyper_p_grids[model][2])]
            for param in grid:
                y_test = y_test_log
                regressor = globals()[str(model)](param)
                fit = regressor.fit(X_train, y_train)
                y_pred = list(fit.predict(X_test))
                y_test = list(y_test)
                
                # get rid of logs
                for j in range(len(y_test)):
                    y_test[j] = 10**y_test[j]
                    y_pred[j] = 10**y_pred[j]
                    
                r2, rmse = get_metrics(y_test, y_pred)
                rmses[i][param]= rmse
                
        elif model == 'LinearRegression':
            grid = [True, False]
            for param in grid:
                y_test = y_test_log
                regressor = globals()[str(model)](param)
                fit = regressor.fit(X_train, y_train)
                y_pred = list(fit.predict(X_test))
                y_test = list(y_test)
                
                #get rid of logs
                for j in range(len(y_test)):
                    y_test[j] = 10**y_test[j]
                    y_pred[j] = 10**y_pred[j]
                    
                r2, rmse = get_metrics(y_test, y_pred)        
                rmses[i][param]= rmse
                
        else:
            grid1 = [j for j in range(hyper_p_grids[model][0][0], hyper_p_grids[model][0][1], \
                                hyper_p_grids[model][0][2])]
            
            grid2 = [j for j in range(hyper_p_grids[model][1][0], hyper_p_grids[model][1][1], \
                                hyper_p_grids[model][1][2])]
            for param in grid1:
                y_test = y_test_log
                rmses[i][param]= {}
                for param2 in grid2:
                    y_test = y_test_log
                    y_pred = list(xgb_fit(X_train, y_train, X_test, [param, param2]))
                    y_test = list(y_test)
            
                    # get rid of logs
                    for j in range(len(y_test)):
                        y_test[j] = 10**y_test[j]
                        y_pred[j] = 10**y_pred[j]
                        
                    r2, rmse = get_metrics(y_test, y_pred) 
                    rmses[i][param][param2] = rmse

    if model != 'BoostedTree':
        best_param = 1e6
        min_rmse =1e6
        for param in grid:
            p_rmses = []
            for i in rmses:
                p_rmses.append(rmses[i][param])
            if np.mean(p_rmses) < min_rmse:
                min_rmse = rmses[i][param]
                best_param = param
        arguments[model] = best_param
        
    else:
        best_params = [1e6, 1e6]
        min_rmse =1e6
        for param in grid1:
            for param2 in grid2:
                p_rmses = []
                for i in rmses:
                    p_rmses.append(rmses[i][param][param2])
                if np.mean(p_rmses) < min_rmse:
                    min_rmse = rmses[i][param][param2]
                    best_param = [param, param2]
                        
        arguments[model] = best_params        

### Get data

In [8]:
pairs = []
for origin in origins:
    for destination in destinations:
        pairs.append((origin, destination))

triples = []
for pair in pairs:
    for response in responses:
        triples.append((pair[0], pair[1], response))
        
# Read in data
groups = {}
for pair in pairs:
    groups[pair] = pd.read_csv('./data/grouped_' + str(pair) + '_gdp.csv', skipinitialspace=True)
    groups[pair] = make_new_features(groups[pair])
    groups[pair]['distance'] = get_distance(coordinates[pair[1]], coordinates[pair[0]], groups[pair].shape[0])
    groups[pair] = groups[pair][all_variables]

/Users/jasonterry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


### Set up models and metrics

In [9]:
models = {'LinearRegression': LinearRegression, 'BayesianRidge': BayesianRidge, \
          'RandomForestRegressor': RandomForestRegressor, 'KNeighborsRegressor': KNeighborsRegressor,\
          'Baseline':'Basic Line Fit', 'BoostedTree': 'XGBoost Boosted Tree'}
r2s = {}
rmses = {}

# fit intercept, n_interations, n_estimators, num_neighbors, (max_depth, n_estimators)
arguments = {'LinearRegression': True, 'BayesianRidge': 300, 'RandomForestRegressor': 10,\
             'KNeighborsRegressor': 6,  'Baseline': None, 'BoostedTree': [6, 10]}


# param range, step size    
hyper_p_grids = {'LinearRegression': [True, False], 'BayesianRidge': [200, 400, 2], \
                 'RandomForestRegressor': [10, 40, 2],'KNeighborsRegressor': [4, 20, 2], \
                 'BoostedTree': [[3, 10, 1], [5, 30, 5]]}

# Make metric dictionaries
r2s = {}
rmses = {}
predictions = {}
trues = {}
results = {}
for y in responses:
    r2s[y] = {}
    rmses[y] = {}
    predictions[y] = {}
    trues[y] = {}
    results[y] = {}
    for pair in pairs:
        r2s[y][pair] = {}
        rmses[y][pair] = {}
        predictions[y][pair] = {}
        results[y][pair] = {}

average_rmses = {}
median_rmses = {}
for model in models:
    average_rmses[model] = {}
    median_rmses[model] = {}
    for response in responses:
        average_rmses[model][response] = 0
        median_rmses[model][response] = []

#### XGBoost Model

In [10]:
def xgb_fit(X_train, y_train, X_test, params = arguments['BoostedTree']):
    '''Regression using XGBoost'''
    
    max_depth = int(params[0])
    n_estimators = int(params[1])
    
    xg_reg = xgb.XGBRegressor(max_depth = max_depth, n_estimators = n_estimators)
    xg_reg.fit(X_train,y_train)

    y_pred = xg_reg.predict(X_test)
    
    return y_pred

### Run models

In [ ]:
# Go through every combination of countries
for pair in pairs:
    print('\n \n', pair)
    
    # Go through each model
    for model in models:
        print('\n', model)
        global this_model
        this_model = models[model]
        X = groups[pair][features]
        X_train = X.iloc[:int(0.8*X.shape[0]), :]
        X_test = X.iloc[int(0.8*X.shape[0]):, :]
        
        # Go through each response variable
        for response in responses:
            print(response)
            y = np.log10(groups[pair][response] + 1e-8)
            y_train = y.iloc[:int(0.8*y.shape[0])]
            y_test = y.iloc[int(0.8*y.shape[0]):]
            
            # search for hyperparameters using grid cross validation
            if model != 'Baseline':
                hyper_search(X_train, y_train, model)

            if model != 'Baseline' and model != 'BoostedTree':
                regressor = globals()[str(model)](arguments[model])
                fit = regressor.fit(X_train, y_train)
                y_pred = list(fit.predict(X_test))

            elif model == 'Baseline':
                y_pred = []
                for index, row in X_test.iterrows():
                    m = np.log10(row.last_month + 1e-8) - np.log10(row.two_months_ago + 1e-9)
                    if response == 'Value':
                        y_pred.append(np.log10(row.last_month + 1e-8) + m)
                    elif response == 'next_month':
                        y_pred.append(np.log10(row.last_month + 1e-8) + 2*m)
                    else:
                        y_pred.append(np.log10(row.last_month + 1e-8) + 3*m)

            else:
                y_pred = list(xgb_fit(X_train, y_train, X_test, arguments[model]))
#             print('Model finished')

            y_test = list(y_test)

            # get rid of logs
            for i in range(len(y_test)):
                y_test[i] = 10**y_test[i]
                y_pred[i] = 10**y_pred[i]

            r2, rmse = get_metrics(y_test, y_pred)
            r2s[response][pair][model] = r2
            rmses[response][pair][model] = rmse
            predictions[response][pair][model] = list(y_pred)
            trues[response][pair] = list(y_test)
            average_rmses[model][response] += rmse
            median_rmses[model][response].append(rmse)
            results[response][pair][model] = [y_pred, y_test]



 
 ('DR_Congo', 'USA')

 LinearRegression
Value
next_month
two_months_later

 BayesianRidge
Value
next_month
two_months_later

 RandomForestRegressor
Value
next_month
two_months_later

 KNeighborsRegressor
Value
next_month
two_months_later

 Baseline
Value
next_month
two_months_later

 BoostedTree
Value
next_month
two_months_later

 
 ('DR_Congo', 'UK')

 LinearRegression
Value
next_month
two_months_later

 BayesianRidge
Value
next_month
two_months_later

 RandomForestRegressor
Value
next_month
two_months_later

 KNeighborsRegressor
Value
next_month
two_months_later

 Baseline
Value
next_month
two_months_later

 BoostedTree
Value
next_month
two_months_later

 
 ('DR_Congo', 'France')

 LinearRegression
Value
next_month
two_months_later

 BayesianRidge
Value
next_month
two_months_later

 RandomForestRegressor
Value
next_month
two_months_later

 KNeighborsRegressor
Value
next_month
two_months_later

 Baseline
Value
next_month
two_months_later

 BoostedTree
Value
next_month
two_months_la

In [21]:
min_avg = 1e6
min_model = ''
for model in average_rmses:
    for response in responses:
        average_rmses[model][response] = np.mean(average_rmses[model][response])
        print('Average RMSE for %s for response %s is %.4f' %(model, response, average_rmses[model][response]))
        if average_rmses[model][response] < min_avg:
            min_avg = average_rmses[model][response]
            min_model = model
    print('')
print('Minimum RMSE average is %.4f using %s' %(min_avg, min_model))

Average RMSE for LinearRegression for response Value is 880483248613635764467328956269295323924664651809560723456.0000
Average RMSE for LinearRegression for response next_month is 2217702180118535970924483781160178469700147699266300674048.0000
Average RMSE for LinearRegression for response two_months_later is 392399838881187488105795327796824134297438482947624665088.0000

Average RMSE for BayesianRidge for response Value is 18291396759557971134710816037395915544647469707034624.0000
Average RMSE for BayesianRidge for response next_month is 15417219100719708011239004209967046452793698114600960.0000
Average RMSE for BayesianRidge for response two_months_later is 1681841723477506358094223420868470011446165621440512.0000

Average RMSE for RandomForestRegressor for response Value is 22856.6458
Average RMSE for RandomForestRegressor for response next_month is 22717.0761
Average RMSE for RandomForestRegressor for response two_months_later is 22488.6218

Average RMSE for KNeighborsRegressor for

In [41]:
min_meds = {'Value':1e6, 'next_month':1e6, 'two_months_later':1e6}
min_med_model = {'Value':None, 'next_month':None, 'two_months_later':None}

median_rmses = {}
for response in responses:
    median_rmses[response] = {}
    for model in models:
        this_model = []
        for pair in pairs:
            this_model.append(rmses[response][pair][model])
        median_rmses[response][model] = np.median(this_model)
        if median_rmses[response][model] < min_meds[response]:
            min_meds[response] = median_rmses[response][model]
            min_med_model[response] = model


In [42]:
min_meds

{'Value': 26.0351,
 'next_month': 28.896099999999997,
 'two_months_later': 26.6033}

In [44]:
min_med_model

{'Value': 'KNeighborsRegressor',
 'next_month': 'RandomForestRegressor',
 'two_months_later': 'KNeighborsRegressor'}

In [39]:
arguments

{'Baseline': None,
 'BayesianRidge': 200,
 'BoostedTree': [1000000.0, 1000000.0],
 'KNeighborsRegressor': 4,
 'LinearRegression': True,
 'RandomForestRegressor': 12}

In [45]:
median_rmses

{'Value': {'Baseline': 140.66395,
  'BayesianRidge': 100.5904,
  'BoostedTree': 32.340900000000005,
  'KNeighborsRegressor': 26.0351,
  'LinearRegression': 419.08445,
  'RandomForestRegressor': 31.446799999999996},
 'next_month': {'Baseline': 410.2502,
  'BayesianRidge': 78.33655,
  'BoostedTree': 37.63805,
  'KNeighborsRegressor': 32.32055,
  'LinearRegression': 453.14885,
  'RandomForestRegressor': 28.896099999999997},
 'two_months_later': {'Baseline': 1665.8959,
  'BayesianRidge': 83.24085,
  'BoostedTree': 30.8804,
  'KNeighborsRegressor': 26.6033,
  'LinearRegression': 152.279,
  'RandomForestRegressor': 29.8373}}

### Visualize results

In [23]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go

import geopandas
from mpl_toolkits.basemap import Basemap
import plotly
import dash
import plotly.offline as ply
from plotly.graph_objs import *
from scipy.io import netcdf
import scipy.stats as stat

In [24]:
quads = []
for pair in pairs:
    for model in models:
        for response in responses:
            quads.append((pair[0], pair[1], response, model, 0))
            quads.append((pair[0], pair[1], response, model, 1))

In [47]:
quad = quads[0]
pair = ("Syria", "Germany")
model = 'LinearRegression'
response = 'Value'
fig = go.FigureWidget(
    data=[dict(
        x = trues[response][pair],
        y = predictions[response][pair][model],
        name = str(quad), 
        mode = 'markers',
        opacity = 0.6
        )
    ]
)
years = [2000 + i for i in range(18)]
marker_places= [12*i for i in range(18)]
tick_labs = [str(year) for year in years]
fig.layout.hovermode = 'closest'

fig.layout.yaxis.title = 'Refugees Taken In (Predicted)'
fig.layout.xaxis.title = 'Refugees Taken In (True)'

In [48]:
from ipywidgets import interact
@interact(dest = destinations, org = origins, time = responses, model = list(models.keys()))
def update_arguments(dest=['Canada'], org = ["Syria"], time = ['Value'], model = ["LinearRegression"]):

    this_pair = (org, dest)

    fig.data[0].x = trues[time][this_pair] \
                     + \
                    np.random.rand(len(trues[time][this_pair]))
        
    fig.data[0].y = predictions[time][this_pair][model] \
                     + \
                    np.random.rand(len(predictions[time][this_pair][model]))

    fig.layout.title = 'Refugees taken in by %s from %s ' %(dest, org)
fig

interactive(children=(Dropdown(description='dest', options=('USA', 'UK', 'France', 'Canada', 'Italy', 'Germany…

FigureWidget({
    'data': [{'mode': 'markers',
              'name': "('DR_Congo', 'USA', 'Value', 'LinearReg…

In [ ]:
plt.legend()